In [1]:
%load_ext autoreload

In [3]:
#main
%autoreload
import pandas as pd
import numpy as np
import datetime as dt
from datetime import datetime
import scipy as sc
from scipy.stats import norm
import math
import utilities as ut
import utilities_2 as ut2

In [4]:
df=pd.read_csv('EUROSTOXX50_2023_Dataset.csv', index_col=0)
#df = df.fillna(method='ffill')
name_stocks0=['ADSGn.DE', 'ALVG.DE', 'MUVGn.DE',
                            'OREP.PA'] 
dates_num=['2016-03-18','2019-03-20']
dates_den = ['2016-03-17','2019-03-19']

np_num, np_den=ut.read_our_CSV(df,name_stocks0, dates_num, dates_den)

In [5]:
alpha=0.95
notional=1e7
delta=1
n_asset=4
weights=np.ones((n_asset,1))/n_asset
returns=np.log(np_num/np_den)

In [6]:
VaR, ES=ut.AnalyticalNormalMeasures(alpha, weights, notional, delta, returns)
VaR, ES

(146813.58536847658, 185434.4799413243)

In [7]:
#CHECK
VaR_check=ut.plausibilityCheck(returns, weights, alpha, notional, delta)
VaR_check

139905.9213679971

In [8]:
#Exercise 2
#Parameters
sett_date='2023-01-31'
expiry='2023-04-05'
strike=25
value_ptf=25870000
volatility=0.154
dividend=0.031
alpha_2=0.99
days_VaR=10
rate=0

In [9]:
#We select only the stocks of Vonovia between the settlement date and 2y before
name_stocks2=['VNAn.DE']
dates_num=['2021-02-01',sett_date]
dates_den = ['2021-01-29','2023-01-30']

np_num2, np_den2=ut.read_our_CSV(df,name_stocks2, dates_num, dates_den)

In [10]:
stockPrice=np_num2[len(np_num2)-1]
numberOfShares=stockPrice*value_ptf
numberOfPuts=numberOfShares
start = datetime.strptime(sett_date, "%Y-%m-%d")
end = datetime.strptime(expiry, "%Y-%m-%d")
diff = end - start
timeToMaturityInYears=diff.days/365
riskMeasureTimeIntervalInYears=days_VaR/365
NumberOfDaysPerYears=260
logReturns=np.log(np_num2/np_den2)

In [ ]:
VaR_MC=ut2.FullMonteCarloVaR(logReturns, numberOfShares, numberOfPuts, stockPrice, strike, rate, dividend,
                            volatility, timeToMaturityInYears, riskMeasureTimeIntervalInYears, alpha, NumberOfDaysPerYears)
VaR_DN=ut2.DeltaNormalVaR(logReturns, numberOfShares, numberOfPuts, stockPrice, strike, rate, dividend,
                            volatility, timeToMaturityInYears, riskMeasureTimeIntervalInYears, alpha, NumberOfDaysPerYears)

In [ ]:
#CHECK
VaR_checkMC=ut.plausibilityCheck(logReturns, 1, alpha_2, value_ptf, riskMeasureTimeIntervalInDay)
VaR_checkDN=ut.plausibilityCheck(logReturns, 1, alpha_2, value_ptf, delta)
VaR_check